In [ ]:
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
import cv2

import torch
import torch.nn as nn
import torchvision.models as models

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

from torchvision.transforms import Compose, Resize, ToTensor
import cv2
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F



In [ ]:
store_names = ["엉생", "명동칼국수", "오리", "참치"]
imgs = []
labels = []
drive_path = '/content/drive'
base_path = f"{drive_path}/MyDrive/TrainFiles/"

In [ ]:
import time

type(time.strftime("%b %d %Y %H:%M:%S", time.localtime()))
# time.strftime('z')

str

In [ ]:
a=123
b=234
f'{a b}'

'(123, 234)'

In [ ]:
# label_encoder = LabelEncoder()
unique_labels = ["엉생", "명동칼국수", "오리", "참치"]  # 고유한 라벨 목록
# label_encoder.fit(unique_labels)

class CustomImageDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_labels = []
        self.img_paths = []

        for label, label_dir in enumerate(os.listdir(root_dir)):
            label_dir_path = os.path.join(root_dir, label_dir)
            if os.path.isdir(label_dir_path):
                for img_file in os.listdir(label_dir_path):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg', 'JPG')):
                        self.img_paths.append(os.path.join(label_dir_path, img_file))
                        label_onehot = F.one_hot(torch.tensor(label), num_classes=4).float()
                        self.img_labels.append(label_onehot)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        # image = cv2.imread(img_path)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # cv2는 BGR 형식으로 이미지를 불러오므로 RGB로 변환
        # image = Image.fromarray(image)
        image = Image.open(img_path).conver("RGB")

        label = self.img_labels[idx]
        # label = label_encoder.transform([label])[0]

        if self.transform:
            image = self.transform(image)

        return image, label

train_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.RandomResizedCrop(480),
    # transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(480),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform = Compose([Resize((256, 256)), ToTensor()])
# train_dataset = CustomImageDataset('/content/drive/MyDrive/TrainFiles/', transform=train_transforms)
train_dataset = CustomImageDataset('/content/drive/MyDrive/TrainFiles/', transform=transform)
# test_dataset = CustomImageDataset('/content/drive/MyDrive/TestFiles/', transform=test_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)



In [ ]:
store_num = 4

resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, store_num)

resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, store_num)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 140MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

In [ ]:
import torch.optim as optim
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.001)

# 장치 설정 (GPU 사용 가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
resnet18 = resnet18.to(device)

In [ ]:
# 학습 루프
num_epochs = 50
for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    if (epoch > 5) and (epoch%3==0):
        torch.save(resnet50.state_dict(), '/content/drive/MyDrive/TrainOutput/'+str(10000+epoch)[1:]+'_resnet50_trained.pth')

Epoch 1/50, Loss: 0.9631306242484313
Epoch 2/50, Loss: 0.6233703933465176
Epoch 3/50, Loss: 0.3793154602440504
Epoch 4/50, Loss: 0.5592366247796096
Epoch 5/50, Loss: 0.1640642373703229
Epoch 6/50, Loss: 0.04286938860343817
Epoch 7/50, Loss: 0.03887602165774203
Epoch 8/50, Loss: 0.09279740645508401
Epoch 9/50, Loss: 0.36302655785440063
Epoch 10/50, Loss: 0.19055283601539066
Epoch 11/50, Loss: 0.15461077708273363
Epoch 12/50, Loss: 0.1824619583128832
Epoch 13/50, Loss: 0.2632060712770535
Epoch 14/50, Loss: 0.046709217328148395
Epoch 15/50, Loss: 0.14715897112947482
Epoch 16/50, Loss: 0.01999344753447729
Epoch 17/50, Loss: 0.13039137793254132
Epoch 18/50, Loss: 0.13170558367318547
Epoch 19/50, Loss: 0.05307323054913491
Epoch 20/50, Loss: 0.07266831633788594
Epoch 21/50, Loss: 0.016955298163781825
Epoch 22/50, Loss: 0.04341519563231253
Epoch 23/50, Loss: 0.1902623071912855
Epoch 24/50, Loss: 0.045318372547626495
Epoch 25/50, Loss: 0.09314347454953047
Epoch 26/50, Loss: 0.05796489445147484


KeyboardInterrupt: 

In [ ]:
import torch.optim as optim
# 손실 함수 및 옵티마이저 설정
criterion18 = nn.CrossEntropyLoss()
optimizer18 = optim.Adam(resnet18.parameters(), lr=0.001)

# 장치 설정 (GPU 사용 가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

In [ ]:
# 학습 루프
num_epochs = 50
for epoch in range(num_epochs):
    resnet50.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer18.zero_grad()
        outputs = resnet18(inputs)
        loss = criterion18(outputs, labels)
        loss.backward()
        optimizer18.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    if (epoch > 5) and (epoch%3==0):
        torch.save(resnet18.state_dict(), f'/content/drive/MyDrive/TrainOutput/{str(10000+epoch)[1:]}_resnet18_trained_{str(running_loss/len(train_loader))[:5]}.pth')

Epoch 1/50, Loss: 0.7396496738760899
Epoch 2/50, Loss: 0.3567675829697878
Epoch 3/50, Loss: 0.26047481121853566
Epoch 4/50, Loss: 0.20096074645885098
Epoch 5/50, Loss: 0.11476503265126106
Epoch 6/50, Loss: 0.12286517134187026
Epoch 7/50, Loss: 0.1584634957417177
Epoch 8/50, Loss: 0.1027880899536495
Epoch 9/50, Loss: 0.060601894808664486
Epoch 10/50, Loss: 0.09410561686411548
Epoch 11/50, Loss: 0.24047646943617088
Epoch 12/50, Loss: 0.10835289934094852
Epoch 13/50, Loss: 0.06093641743138552
Epoch 14/50, Loss: 0.01775893676527537
Epoch 15/50, Loss: 0.0559981979391663
Epoch 16/50, Loss: 0.09487713426176178
Epoch 17/50, Loss: 0.12826062358306864
Epoch 18/50, Loss: 0.04879591596993403
Epoch 19/50, Loss: 0.045642854829053156
Epoch 20/50, Loss: 0.05123882506487485
Epoch 21/50, Loss: 0.020654466244032405
Epoch 22/50, Loss: 0.013386941467522262
Epoch 23/50, Loss: 0.042922272730580656
Epoch 24/50, Loss: 0.04066465804237687
Epoch 25/50, Loss: 0.014641293300090369
Epoch 26/50, Loss: 0.055503497784

KeyboardInterrupt: 

# 양자화

In [ ]:
import torch.quantization
store_num = 4
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, store_num)

trained_state_dict = torch.load('/content/drive/MyDrive/TrainOutput/0030_resnet18_trained_0.003.pth')
model.load_state_dict(trained_state_dict)
model = model.cpu()
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(quantized_model.state_dict(), "/content/drive/MyDrive/TrainOutput/0030_quantized_resnet18.pth")

In [ ]:
quantized_model

In [ ]:
from PIL import Image

store_num = 4

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, store_num)

transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/오리/IMG_0715.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가
print(image.size())
with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 103MB/s]


torch.Size([1, 3, 256, 256])
Predicted Class: 3


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0777.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 3


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0798.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 3


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0822.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 0


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0828.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 0


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0794.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 3


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/엉생/IMG_0786.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 3


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/참치/IMG_0676.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 2


In [ ]:
transform = Compose([Resize((256, 256)), ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/명동칼국수/IMG_0842.JPG')
image = transform(image).unsqueeze(0)  # 배치 차원 추가

with torch.no_grad():
    output = model(image)
    # 결과 처리 (예: 출력 확률에서 최대값을 가진 클래스 찾기)
    _, predicted = torch.max(output, 1)
    print("Predicted Class:", predicted.item())

Predicted Class: 0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Basic Block 정의
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# ResNet18 모델 생성 함수
def resnet18(pretrained=False, num_classes=4, model_path='/content/drive/MyDrive/TrainOutput/0030_quantized_resnet18.pth'):
    model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)
    if pretrained:
        model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
model = resnet18()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:355: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.0.weight", "conv1.1.weight", "conv1.1.bias", "conv1.1.running_mean", "conv1.1.running_var", "conv2_x.0.residual_function.0.weight", "conv2_x.0.residual_function.1.weight", "conv2_x.0.residual_function.1.bias", "conv2_x.0.residual_function.1.running_mean", "conv2_x.0.residual_function.1.running_var", "conv2_x.0.residual_function.3.weight", "conv2_x.0.residual_function.4.weight", "conv2_x.0.residual_function.4.bias", "conv2_x.0.residual_function.4.running_mean", "conv2_x.0.residual_function.4.running_var", "conv2_x.1.residual_function.0.weight", "conv2_x.1.residual_function.1.weight", "conv2_x.1.residual_function.1.bias", "conv2_x.1.residual_function.1.running_mean", "conv2_x.1.residual_function.1.running_var", "conv2_x.1.residual_function.3.weight", "conv2_x.1.residual_function.4.weight", "conv2_x.1.residual_function.4.bias", "conv2_x.1.residual_function.4.running_mean", "conv2_x.1.residual_fu

In [ ]:
torch.load('/content/drive/MyDrive/TrainOutput/0030_quantized_resnet18.pth')

In [ ]:
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/TrainOutput/0030_quantized_resnet18.pth'))

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:355: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "fc.scale", "fc.zero_point", "fc._packed_params.dtype", "fc._packed_params._packed_params". 

In [ ]:
import pickle

to_pickle = {

}

In [ ]:
### build_pack_script.sh

In [ ]:

dev_install () {
    yum -y update
    yum -y upgrade
    amazon-linux-extras enable python3.8
    yum install python3.8
    update-alternatives --install /usr/bin/python python /usr/bin/python3.8 1
    update-alternatives --install /usr/bin/python python /usr/bin/python2.7 1
    update-alternatives --config python
    1
    alias pip='pip3.8'

}

make_virtualenv () {
    pip install virtualenv
    cd /home
    rm -rf env
    python -m virtualenv env --python=python3.8
    source env/bin/activate
}

install_pytorch () {
    pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
}

install_packages () {
    pip install pillow
}

gather_pack () {
    cd /home
    rm -rf lambdapack
    mkdir lambdapack
    cd lambdapack

    # Copy python pakages from virtual environment
    cp -R /home/env/lib/python3.8/site-packages/* .
    cp -R /home/env/lib64/python3.8/site-packages/* .

    echo "Original size $(du -sh /home/lambdapack | cut -f1)"

    # Clean pakages
    find . -type d -name "tests" -exec rm -rf {} +
    find -name "*.so" | xargs strip
    find -name "*.so.*" | xargs strip
    rm -r pip
    rm -r pip-*
    rm -r wheel
    rm -r wheel-*
    rm easy_install.py
    find . -name \*.pyc -delete
    echo "Stripped size $(du -sh /home/lambdapack | cut -f1)"

    # Compress
    zip -FS -r1 /host/pack.zip * > /dev/null
    echo "Compressed size $(du -sh /host/pack.zip | cut -f1)"
}

add_pack () {
    cd /host
    zip -9 -q -r pack.zip lambda_function.py
}

main () {
    dev_install
    make_virtualenv
    install_pytorch
    install_packages
    gather_pack
    add_pack
}

main

In [ ]:
### lambda_function.py

In [ ]:
import boto3
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import json
from torchvision import transforms
import io

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=1000):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output
def resnet18(num_classes=200):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)
model = resnet18(num_classes=4)

ACCESS_KEY = os.environ.get('ACCESS_KEY')
SECRET_KEY = os.environ.get('SECRET_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')
max_length = os.environ.get('max_length') # 20
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)

# Load preprocessing parameters
if not os.path.isfile('/tmp/params.pkl'):
    s3_client.download_file(BUCKET_NAME, 'params.pkl', '/tmp/params.pkl')
with open('/tmp/params.pkl', 'rb') as pkl:
    params = pickle.load(pkl)

# Check if models are available
# Download model from S3 if model is not already present
if not os.path.isfile('/tmp/model.pth'):
    s3_client.download_file(BUCKET_NAME, 'model.pth', '/tmp/model.pth')

model.load_state_dict(torch.load("/tmp/model.pth"))
model.eval()

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
image = Image.open('/content/drive/MyDrive/TrainFiles/오리/IMG_0715.JPG')
image = transform(image).unsqueeze(0)




def lambda_handler(event, context):
    if 'body' in event:
        image_base64 = event['body']
        image_data = base64.b64decode(image_base64)

        # 이미지 데이터를 PIL 이미지 객체로 변환
        image = Image.open(io.BytesIO(image_data))
        image = transform(image).unsqueeze(0)

        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
            response = {'predicted_class': predicted.item()}
        return {
            "statusCode" : 200,
            "headers" : {"Contetn-Type" : "application/json"},
            "body" : json.dumps(response)
        }
    else:
        return {
            'statusCode': 400,
            'body': json.dumps('No image data found')
        }

In [ ]:

dev_install () {
    yum -y update
    yum -y upgrade
    amazon-linux-extras enable python3.8
    yum install -y python3.8
    update-alternatives --install /usr/bin/python python /usr/bin/python3.8 1
    update-alternatives --install /usr/bin/python python /usr/bin/python2.7 1
    update-alternatives --config python
    1
    alias pip='pip3.8'

}

make_virtualenv () {
    pip install virtualenv
    cd /home
    rm -rf env
    python -m virtualenv env --python=python3.8
    source env/bin/activate
}

install_pytorch () {
    pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
}

install_packages () {
    pip install pillow
}

gather_pack () {
    cd /home
    rm -rf lambdapack
    mkdir lambdapack
    cd lambdapack

    # Copy python pakages from virtual environment
    cp -R /home/env/lib/python3.8/site-packages/* .
    cp -R /home/env/lib64/python3.8/site-packages/* .

    echo "Original size $(du -sh /home/lambdapack | cut -f1)"

    # Clean pakages
    find . -type d -name "tests" -exec rm -rf {} +
    find -name "*.so" | xargs strip
    find -name "*.so.*" | xargs strip
    rm -r pip
    rm -r pip-*
    rm -r wheel
    rm -r wheel-*
    rm easy_install.py
    find . -name \*.pyc -delete
    echo "Stripped size $(du -sh /home/lambdapack | cut -f1)"

    # Compress
    zip -FS -r1 /host/pack.zip * > /dev/null
    echo "Compressed size $(du -sh /host/pack.zip | cut -f1)"
}

add_pack () {
    cd /host
    zip -9 -q -r pack.zip lambda_function.py
}

main () {
    dev_install
    make_virtualenv
    install_pytorch
    install_packages
    gather_pack
    add_pack
}

main

In [ ]:
import boto3
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import json
from torchvision import transforms
import io
import base64

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=1000):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output
def resnet18(num_classes=200):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)
model = resnet18(num_classes=4)


BUCKET_NAME = os.environ.get('BUCKET_NAME')
MODEL_NAME = os.environ.get('MODEL_NAME')
s3_client = boto3.client('s3')

# Load preprocessing parameters
# if not os.path.isfile('/tmp/params.pkl'):
#     s3_client.download_file(BUCKET_NAME, 'params.pkl', '/tmp/params.pkl')
# with open('/tmp/params.pkl', 'rb') as pkl:
#     params = pickle.load(pkl)

# Check if models are available
# Download model from S3 if model is not already present
if not os.path.isfile('/tmp/model.pth'):
    s3_client.download_file(BUCKET_NAME, 'model.pth', '/tmp/model.pth')

model.load_state_dict(torch.load("/tmp/model.pth"))
model.eval()

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
# image = Image.open('/content/drive/MyDrive/TrainFiles/오리/IMG_0715.JPG')
# image = transform(image).unsqueeze(0)




def lambda_handler(event, context):
    if 'body' in event:
        image_base64 = event['body']
        image_data = base64.b64decode(image_base64)

        # 이미지 데이터를 PIL 이미지 객체로 변환
        image = Image.open(io.BytesIO(image_data))
        image = transform(image).unsqueeze(0)

        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
            response = {'predicted_class': predicted.item()}
        return {
            "statusCode" : 200,
            "headers" : {"Contetn-Type" : "application/json"},
            "body" : json.dumps(response)
        }
    else:
        return {
            'statusCode': 400,
            'body': json.dumps('No image data found')
        }